In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import trange
import multiprocessing
import re

In [2]:
from tqdm import tqdm_notebook as tqdm

#### I am gonna play with this code later

import scholarly as sh

ejem = sh.search_pubs_query(batch_5[5].title.text) 

pub = next(ejem)

print(pub)

In [3]:
# Define the query for the ArXiv API

quantum_query = "http://export.arxiv.org/api/query?search_query=cat:quant-ph&start={}&max_results=10000&sortBy=lastUpdatedDate&sortOrder=descending&"


In [4]:
def batch(n):
    globals()['url_batch'+str(n)] = quantum_query.format((n-1)*10000)
    
    
    globals()['batch_'+str(n)] = BeautifulSoup(requests.get(globals()['url_batch'+str(n)]).content, 'lxml').find_all('entry')
    
    while len(globals()['batch_'+str(n)]) <10000:
        print('The batch number {} failed to extract all data.'.format(n))
        print('We will wait 2 mins to try again.')
        time.sleep(120)
        print('2 mins have passed, we will try to retrieve the data again.\n')
        
        globals()['batch_'+str(n)] = BeautifulSoup(requests.get(globals()['url_batch'+str(n)]).content, 'lxml').find_all('entry')
    
    return "We have retrive all data for bacth number {}".format(n)

In [ ]:
%%time

for i in trange(1,11):
    batch(i)

In [ ]:
# I as able to only get the 5,000 most recent papers

for i in range(5):
    print(len(globals()['batch_'+str(i+1)]))

<br>
<br>
<br>

# Here I start the cosntruction of the Data Frame.

## All of the following cells create and update the dataframe property by proterpty.

In [ ]:
quantum_df = pd.DataFrame([p.title.text for p in batches_1_5], columns=['title'])

In [ ]:
quantum_df.head()

In [ ]:
quantum_df['published'] = pd.DataFrame([p.published.text for p in batches_1_5])

In [ ]:
quantum_df.head()

In [ ]:
quantum_df['summary'] = pd.DataFrame([p.summary.text for p in batches_1_5])

In [ ]:
quantum_df.head()

In [ ]:
quantum_df['num authors'] = pd.DataFrame([len(a.find_all('author')) for a in batches_1_5])

In [ ]:
quantum_df.head()

In [ ]:
quantum_df.head()

In [ ]:
categories = list(map(lambda x: [y['term'] for y in x], [c for x in batches_1_5[:] for c in [x.find_all('category')] if 'term' in str(x)]))

In [ ]:
pd.DataFrame([[x] for x in categories[:10]])

In [ ]:
quantum_df['categories'] = pd.DataFrame([[x] for x in categories])

In [ ]:
quantum_df.head()

In [5]:
# Get the number of pages.

def num_pages(entry):
    if re.search('\d+ pages', str(entry)):
        return re.search('\d+ pages', str(entry)).group()
    else:
        return "Not specified"

In [6]:
# Get the number of figures.

def num_figures(entry):
    if re.search('\d+ figures', str(entry)):
        return re.search('\d+ figures', str(entry)).group()
    else:
        return "Not specified"

In [ ]:
quantum_df['num pages'] = pd.DataFrame(list(map(num_pages, batches_1_5)))

In [ ]:
quantum_df.head()

In [ ]:
quantum_df['num figures'] = pd.DataFrame(list(map(num_figures, batches_1_5)))
quantum_df.head()

# At this point I had to save the data frame to a csv. Note that I also saved the raw list of batches.

In [ ]:
quantum_df.to_csv('quantum_papers.csv')

In [ ]:
pd.DataFrame(batches_1_5).to_csv('batches_1_5.csv')